In [1]:
import logging
import os
import time
import zipfile

import boto3
from botocore.exceptions import ClientError
import requests
from pyspark.sql import SparkSession, types
from pyspark.sql import functions as F

In [2]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
symbol = 'ADAUSDT'
landing_date = '2025-09-27'
PROJECT_PREFIX_UNDERSCORE = os.getenv('PROJECT_PREFIX_UNDERSCORE')
DATA_LAKE_BUCKET = os.getenv('DATA_LAKE_BUCKET')

In [4]:
s3 = boto3.client(
    "s3",
    endpoint_url="http://localhost:9000",
    aws_access_key_id="minio",
    aws_secret_access_key="minio123",
    region_name="us-east-1"  # required by boto3 even for MinIO
)


In [5]:
def download_file(url, file_path):
    if os.path.exists(file_path):
        logger.info(f"{file_path} exists")
        return
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(file_path, "wb") as f:
            for chunk in r.iter_content(chunk_size=1024 * 1024):
                if chunk:
                    f.write(chunk)
        logger.info(
            f"Downloaded {file_path} {(os.path.getsize(file_path) / (1024 * 1024)):.2f}MB"
        )


def remove_file(file_path):
    if os.path.exists(file_path):
        os.remove(file_path)
        logger.info(f"{file_path} removed")


def extract_file(extract_dir, zip_path):
    if not os.path.exists(zip_path):
        logger.info(f"{zip_path} not found")
        return None
    if not os.path.exists(extract_dir):
        os.makedirs(extract_dir)
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(extract_dir)
        file_path = os.path.join(extract_dir, os.listdir(extract_dir)[0])
        logger.info(f"Extracted CSV: {file_path}")
        return file_path


def upload_to_s3(bucket_name, file_path):
    if not bucket_exists(bucket_name):
        s3.create_bucket(Bucket=bucket_name)
    s3_key = f"raw_zone/{os.path.basename(file_path)}"
    s3.upload_file(file_path, bucket_name, s3_key)
    logger.info(f"Uploaded to s3://{bucket_name}/{s3_key} (MinIO)")
    return f"s3a://{bucket_name}/{s3_key}"

def bucket_exists(bucket_name: str) -> bool:
    try:
        s3.head_bucket(Bucket=bucket_name)
        return True
    except ClientError as e:
        if int(e.response["Error"]["Code"]) == 404:
            return False
        raise

In [6]:
script_dir = "/tmp/data/raw"
extract_dir = os.path.join(script_dir, "unzipped_data")
if not os.path.exists(extract_dir):
    os.makedirs(extract_dir)
url = f"https://data.binance.vision/data/spot/daily/aggTrades/{symbol}/{symbol}-aggTrades-{landing_date}.zip"
zip_path = os.path.join(script_dir, url.split("/")[-1])

logger.info(f"Downloading {url} -> {zip_path}")

start_t = time.time()
download_file(url, zip_path)
csv_path = extract_file(extract_dir, zip_path)
end_t = time.time()
logger.info(f"Download + Extract processed in {(end_t - start_t):.3f} sec")

INFO:__main__:Downloading https://data.binance.vision/data/spot/daily/aggTrades/ADAUSDT/ADAUSDT-aggTrades-2025-09-27.zip -> /tmp/data/raw/ADAUSDT-aggTrades-2025-09-27.zip
INFO:__main__:/tmp/data/raw/ADAUSDT-aggTrades-2025-09-27.zip exists
INFO:__main__:Extracted CSV: /tmp/data/raw/unzipped_data/ADAUSDT-aggTrades-2025-09-27.csv
INFO:__main__:Download + Extract processed in 0.010 sec


In [7]:
spark = (
    SparkSession.builder.appName("LandingZoneLocal")
    .config("spark.master", "spark://localhost:7077")
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000")
    .config("spark.hadoop.fs.s3a.access.key", "minio")
    .config("spark.hadoop.fs.s3a.secret.key", "minio123")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    .getOrCreate()
)

25/12/01 18:42:28 WARN Utils: Your hostname, Nguyens-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.22 instead (on interface en0)
25/12/01 18:42:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/01 18:42:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [10]:
s3_url = upload_to_s3(DATA_LAKE_BUCKET, csv_path)

INFO:__main__:Uploaded to s3://crypto-cloud-dev-650251698703-data-lake-bucket/raw_zone/ADAUSDT-aggTrades-2025-09-27.csv (MinIO)


In [9]:
schema = types.StructType(
    [
        types.StructField("agg_trade_id", types.LongType(), True),
        types.StructField("price", types.DoubleType(), True),
        types.StructField("quantity", types.DoubleType(), True),
        types.StructField("first_trade_id", types.LongType(), True),
        types.StructField("last_trade_id", types.LongType(), True),
        types.StructField("timestamp", types.LongType(), True),
        types.StructField("is_buyer_maker", types.BooleanType(), True),
        types.StructField("is_best_match", types.BooleanType(), True),
    ]
)

df = spark.read.option("header", "false").schema(schema).csv(s3_url)

25/12/01 18:43:00 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: s3a://crypto-cloud-dev-650251698703-data-lake-bucket/raw_zone/ADAUSDT-aggTrades-2025-09-27.csv.
java.lang.RuntimeException: java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2688)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3431)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:53)
	at o

Py4JJavaError: An error occurred while calling o44.csv.
: java.lang.RuntimeException: java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2688)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3431)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.datasources.DataSource$.$anonfun$checkAndGlobPathIfNecessary$1(DataSource.scala:724)
	at scala.collection.immutable.List.map(List.scala:293)
	at org.apache.spark.sql.execution.datasources.DataSource$.checkAndGlobPathIfNecessary(Dat